## Workflow plan

 - Create a list of SMARTS strings for the bond types reported in reference 10.1021/ed085p532 (Tables 2).
 - Develop an RDKit-based query tool that checks whether the current SDF file contains the molecules with specified bond type.
 - If a given bond type is found in the molecule, the constitutive correction for that species will be included.
 - Constitutive corrections are added along atomic Pascal constants (procedure for unknown molecule).

### Materials:
 - [Substructure Filtering in RDKit ](https://www.youtube.com/watch?v=Z1PrErlmTGI)
 - [SMARTS notation](https://en.wikipedia.org/wiki/SMILES_arbitrary_target_specification)

## Compounds loader

In [187]:
from pathlib import Path
from typing import Any

import pytest
from rdkit import Chem
from rdkit.Chem import Mol, MolToSmarts, RemoveHs

from src import DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR
from src.constants.common_molecules import COMMON_MOLECULES
from src.core.compound import MBCompound
from src.core.molecule import MBMolecule
from src.loader import SDFLoader


### Constitutive corrections for relevent bond types

In [188]:
RELEVENT_BOND_TYPES = {
    # There are additional aromatic rings that were not listed in the article by Bain et al. For these rings, constitutive corrections cannot be calculated reliably.
    # SMARTS notations for the bonds C=N, C=C, and N=N must be written explicitly to prevent inclusion of constitutive corrections for these bonds when they are part of an aromatic ring.
    "C=C": {
        "SMARTS": "[C;!$([c]);!$([C]-[c]);!$(C1=CCCCC1);!$([C;!c]=[C;!c]-[C;!c]=[C;!c]);!$([C;X3;H2]=[C;X3;H1]-[$([C;X4;H2]),$([C;X4;H3])])]=[C;!$([c]);!$([C]-[c]);!$(C1=CCCCC1);!$([C;!c]=[C;!c]-[C;!c]=[C;!c]);!$([C;X3;H2]=[C;X3;H1]-[$([C;X4;H2]),$([C;X4;H3])])]",  # C are NOT aromatic theirself and not bound to further aromatic C  - this excludes aromatic rings that are not listed; cyclohexene omitted; excludes C=C-C=C and H2C=CH-CH2- group; all exclusions must be applied for both atoms
        "constitutive_corr": 5.5,
        "sdf_files": ["C2H4.sdf"],
    },
    "C#C": {
        "SMARTS": "[C;!$([C]-[c]);!$([C]([C])#[C]-[C](=[O])-[C])]#[C;!$([C]-[c]);!$([C]([C])#[C]-[C](=[O])-[C])]",  # C#C but without aromatic [c] neighbours; excluded RC#C-C(=O)R; all exclusions must be applied for both atoms
        "constitutive_corr": 0.8,
        "sdf_files": ["C2H2.sdf"],
    },
    "C=C-C=C": {
        "SMARTS": "[C;!$([c])]=[C;!$([c])]-[C;!$([c])]=[C;!$([c])]",  # For conjugated bond systems having at least 3 C=C bonds, constitutive corrections for both C=C and C=C-C=C will be added.
        "constitutive_corr": 10.6,
        "sdf_files": ["C=C-C=C.sdf"],
    },
    "Ar-C#C-Ar": {
        "SMARTS": "[c]-[#6;X2]#[#6;X2]-[c]",
        "constitutive_corr": 3.85,
        "sdf_files": ["Ar-C#C-Ar.sdf"],
    },
    "CH2=CH-CH2-": {
        "SMARTS": "[C;X3;H2]=[C;X3;H1]-[$([C;X4;H2]),$([C;X4;H3])]",
        "constitutive_corr": 4.5,
        "sdf_files": ["allyl_group.sdf"],
    },
    "C=O": {
        "SMARTS": "[C;X3;!$([C]-[c]);!$([C](=[O;X1])[O*]);!$([C](=[O;X1])[N*]);!$([C]([C]#[C]-[C])(=[O])[C])]=[O;X1]",  # Exclude C=O groups attached to aromatic carbons; omitted additional bond to O/N in any form.
        "constitutive_corr": 6.3,
        "sdf_files": ["C=O.sdf"],
    },
    "RCOOH": {
        "SMARTS": "[C;X3;!$([C]-[c])](=[O;X1])[$([O;H1;X2]),$([O-;X1])]",  # IMPORTANT! Both RCOO- and RCOOH groups will be matched. Loosening of some SMARTS conditions will enhance accuarcy of the final molecule diamag. This must be noted in Software's MANUAL.
        "constitutive_corr": -5.0,
        "sdf_files": ["RCOOH.sdf", "RCOO-.sdf"],
    },
    "RCOOR": {
        "SMARTS": "[C;X3;!$([C]-[c])](=[O;X1])[O;X2]-[C]",
        "constitutive_corr": -5.0,
        "sdf_files": ["RCOOR.sdf"],
    },
    "C(=O)NH2": {
        "SMARTS": "[C;X3;!$([C]-[c])](=[O;X1])[$([N;X3;H2]),$([N;X3;H1][C])]",  # Intentional extention for considering not only RCONH2, but also RCONHR. This must be noted in Software's MANUAL.
        "constitutive_corr": -3.5,
        "sdf_files": ["RCONH2.sdf", "RCONHR.sdf"],
    },
    "N=N": {
        "SMARTS": "[N;X2;!n]=[N;X2;!n]",  # N are NOT aromatic - this excludes aromatic rings that are not listed. Azides R-N=N(+)=N(-) were omitted. This must be noted in Software's MANUAL.
        "constitutive_corr": 1.85,
        "sdf_files": ["N=N.sdf"],
    },
    "C=N": {
        "SMARTS": "[C;X2,X3;!$([c])]=[N;!$([n]);!$([N](=[C]([C])[C])-[N]=[C]([C])[C])]",  # N and C are NOT aromatic - this excludes aromatic rings that are not listed. Carbon with two or three neighbours (cases C=C=N- and Any-C=N-); excludes bond type R2C=N–N=CR2
        "constitutive_corr": 8.15,
        "sdf_files": ["C=N.sdf", "R2C=N-N=CRAr.sdf"],
    },
    "-N#C": {
        "SMARTS": "[N+;X2]#[C-;X1]",  # Covers isocyanide R-N(+)#C(-) resonance structure.
        "constitutive_corr": 0.0,
        "sdf_files": ["-N#C.sdf"],
    },
    "-C#N": {
        "SMARTS": "[C;X2]#[N;X1]",
        "constitutive_corr": 0.8,
        "sdf_files": ["-C#N.sdf"],
    },
    "N=O": {
        "SMARTS": "[N;X2;!$([N+])]=[O;X1]",  # Formal charge [N+] of nitrogen is not considered, which rejects the -NO2 group and other fragments with N in oxidation state (V).
        "constitutive_corr": 1.7,
        "sdf_files": ["N=O.sdf"],
    },
    "-NO2": {
        "SMARTS": "[C;!c]-[N+;X3]([O-;X1])=[O;X1]",
        "constitutive_corr": -2.0,
        "sdf_files": ["-NO2.sdf"],
    },
    "C-Cl": {
        "SMARTS": "[C;!$([c]);!$([C]([C])([C])([Cl])[Cl]);!$([C;H1]([C])([Cl])[Cl]);!$([C]([C])([C])([Cl])-[C]([C])([C])[Cl])]-[Cl]",  # Rejected bond types: R2CCl2, RCHCl2 and Cl-CR2-CR2-Cl
        "constitutive_corr": 3.1,
        "sdf_files": ["R2CCl2.sdf"],
    },
    "Cl-CR2-CR2-Cl": {
        "SMARTS": "[C;X4]([C])([C])([Cl])-[C;X4]([C])([C])[Cl]",
        "constitutive_corr": 4.3,
        "sdf_files": ["Cl-CR2-CR2-Cl.sdf"],
    },
    "R2CCl2": {
        "SMARTS": "[C;X4]([C])([C])([Cl])[Cl]",
        "constitutive_corr": 1.44,
        "sdf_files": ["R2CCl2.sdf"],
    },
    "RCHCl2": {
        "SMARTS": "[C;X4;H1]([C])([Cl])[Cl]",
        "constitutive_corr": 6.43,
        "sdf_files": ["RCHCl2.sdf"],
    },
    "C-Br": {
        "SMARTS": "[C;!$([c]),!$([C]([C])([C])([Br])-[C]([C])([C])[Br])]-[Br]",  # excluding Br-CR2-CR2-Br bond type
        "constitutive_corr": 4.1,
        "sdf_files": ["C-Br.sdf"],
    },
    "Br-CR2-CR2-Br": {
        "SMARTS": "[C;X4]([C])([C])([Br])-[C;X4]([C])([C])[Br]",
        "constitutive_corr": 6.24,
        "sdf_files": ["Br-CR2-CR2-Br.sdf"],
    },
    "C-I": {
        "SMARTS": "[C;!c]-[I]",
        "constitutive_corr": 4.1,
        "sdf_files": ["C-I.sdf"],
    },
    "Ar-OH": {
        "SMARTS": "[c]-[$([O;X2;H1]),$([O-;X1])]",  # Both Ar-OH and Ar-O(-) protonation states accepted. Must be stated in MANUAL.
        "constitutive_corr": -1,
        "sdf_files": ["Ar-OH.sdf", "Ar-O-.sdf"],
    },
    "Ar-NR2": {
        "SMARTS": "[c]-[N;X3]([C])[C]",
        "constitutive_corr": 1,
        "sdf_files": ["Ar-NR2.sdf"],
    },
    "Ar-C(=O)R": {
        "SMARTS": "[c]-[C;X3](=[O;X1])[C]",
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-COR.sdf"],
    },
    "Ar-COOR": {
        "SMARTS": "[c]-[C;X3](=[O;X1])[O;X2]-[C]",
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-COOR.sdf"],
    },
    "Ar-C=C": {
        "SMARTS": "[c]-[C;X3;!c]=[C;!c]",
        "constitutive_corr": -1.00,
        "sdf_files": ["Ar-C=C.sdf"],
    },
    "Ar-C#C": {
        "SMARTS": "[c]-[C;X2]#[C;X2;!$([C]-[c])]",  # excludes Ar-C#C-Ar bond type
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-C#C.sdf"],
    },
    "Ar-OR": {
        "SMARTS": "[c]-[O;X2][C]",
        "constitutive_corr": -1,
        "sdf_files": ["Ar-OR.sdf"],
    },
    "Ar-CHO": {
        "SMARTS": "[c]-[C;X3;H1]=[O;X1]",
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-CHO.sdf"],
    },
    "Ar-Ar": {
        "SMARTS": "[c]-[c]",  # RDKit recognize that within aromatic ring there is no single bond c-c.
        "constitutive_corr": -0.5,
        "sdf_files": ["Ar-CHO.sdf"],
    },
    "Ar-NO2": {
        "SMARTS": "[c]-[N+;X3](=[O;X1])[O-;X1]",
        "constitutive_corr": -0.5,
        "sdf_files": ["Ar-NO2.sdf"],
    },
    "Ar-Br": {
        "SMARTS": "[c]-[Br]",
        "constitutive_corr": -3.5,
        "sdf_files": ["Ar-Br.sdf"],
    },
    "Ar-Cl": {
        "SMARTS": "[c]-[Cl]",
        "constitutive_corr": -2.5,
        "sdf_files": ["Ar-Cl.sdf"],
    },
    "Ar-I": {
        "SMARTS": "[c]-[I]",
        "constitutive_corr": -3.5,
        "sdf_files": ["Ar-I.sdf"],
    },
    "Ar-COOH": {
        "SMARTS": "[c]-[C;X3](=[O;X1])[$([O;H1;X2]),$([O-;X1])]",  # Both ArCOO- and ArCOOH groups will be matched. This must be noted in Software's MANUAL.
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-COOH.sdf", "Ar-COO-.sdf"],
    },
    "Ar-C(=O)NH2": {
        "SMARTS": "[c]-[C;X3](=[O;X1])[$([N;X3;H2]),$([N;X3;H1][C])]",  # Intentional extention for considering not only ArCONH2 and ArCONHR. This must be noted in MANUAL.
        "constitutive_corr": -1.5,
        "sdf_files": ["Ar-CONH2.sdf", "Ar-CONHR.sdf"],
    },
    "R2C=N-N=CR2": {
        "SMARTS": "[C;X3;!c]([C])([C])=[N;X2;!n]-[N;X2;!n]=[C;X3;!c]([C])[C]",
        "constitutive_corr": 10.2,
        "sdf_files": ["R2C=N-N=CR2.sdf"],
    },
    "RC#C-C(=O)R": {
        "SMARTS": "[C;X2]([C])#[C;X2]-[C;X3](=[O;X1])[C]",
        "constitutive_corr": 0.8,
        "sdf_files": ["RC#C-C(=O)R.sdf"],
    },
    "benzene": {
        "SMARTS": "c1ccccc1",
        "constitutive_corr": -1.4,
        "sdf_files": ["benzene.sdf"],
    },
    "cyclobutane": {
        "SMARTS": "C1CCC1",
        "constitutive_corr": 7.2,
        "sdf_files": ["cyclobutane.sdf"],
    },
    "cyclohexane": {
        "SMARTS": "C1CCCCC1",
        "constitutive_corr": 3.0,
        "sdf_files": ["cyclohexane.sdf"],
    },
    "cyclohexene": {
        "SMARTS": "C1CC=CCC1",  # The cyclohexadiene ring was excluded for simplicity of the SMARTS notation. The constitutive corr for this ring is very close to the sum for two C=C bonds. Must be stated in MANUAL.
        "constitutive_corr": 6.9,
        "sdf_files": ["cyclohexene.sdf"],
    },
    "cyclopentane": {
        "SMARTS": "C1CCCC1",
        "constitutive_corr": 0.0,
        "sdf_files": ["cyclopentane.sdf"],
    },
    "cyclopropane": {
        "SMARTS": "C1CC1",
        "constitutive_corr": 7.2,
        "sdf_files": ["cyclopropane.sdf"],
    },
    "dioxanes": {
        "SMARTS": "[$(O1CCCCO1),$(O1CCOCC1),$(O1CCCOC1)]",  # Assumes that all dioxane isomers have the same constitutive corr. This must be noted in Software's MANUAL.
        "constitutive_corr": 5.5,
        "sdf_files": ["1,2-dioxane.sdf", "1,3-dioxane.sdf", "1,4-dioxane.sdf"],
    },
    "furan": {
        "SMARTS": "o1cccc1",
        "constitutive_corr": -2.5,
        "sdf_files": ["furan.sdf"],
    },
    "imidazole": {
        "SMARTS": "[$(n1cncc1),$([n-]1cncc1),$(n1c[n-]cc1),$([n+]1cncc1),$(n1c[n+]cc1)]",  # all imidazole protonation states are included. This must be noted in Software's MANUAL.
        "constitutive_corr": 8.0,
        "sdf_files": ["imidazole.sdf", "imidazole-.sdf", "imidazole+.sdf"],
    },
    "isoxazole": {
        "SMARTS": "o1nccc1",
        "constitutive_corr": 1.0,
        "sdf_files": ["isoxazole.sdf"],
    },
    "morpholine": {
        "SMARTS": "[$(O1CCNCC1),$(O1NCCCC1),$(O1CNCCC1)]",
        "constitutive_corr": 5.5,  # Assumes the same constant for all 6-membered morpholine isomers. This must be noted in Software's MANUAL.
        "sdf_files": ["1,2-morpholine.sdf", "1,3-morpholine.sdf", "1,4-morpholine.sdf"],
    },
    "piperazine": {
        "SMARTS": "[$(N1CCNCC1),$([N+]1CCNCC1),$([N-]1CCNCC1)]",
        "constitutive_corr": 7.0,
        "sdf_files": ["piperazine.sdf", "piperazine-.sdf", "piperazine+.sdf"],
    },
    "piperidine": {
        "SMARTS": "[$(N1CCCCC1),$([N-]1CCCCC1),$([N+]1CCCCC1)]",  # Assumes the same constant for piperidine at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": 3.0,
        "sdf_files": ["piperidine.sdf", "piperidine-.sdf", "piperidine+.sdf"],
    },
    "pyrazine": {
        "SMARTS": "[$(n1ccncc1),$([n+]1ccncc1),$(n1cc[n+]cc1)]",  # Assumes the same constant for pyrazine at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": 9.0,
        "sdf_files": ["pyrazine.sdf", "pyrazine+.sdf"],
    },
    "pyrimidine": {
        "SMARTS": "[$(n1cnccc1),$([n+]1cnccc1),$(n1c[n+]ccc1)]",  # Assumes the same constant for pyrimidine at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": 6.5,
        "sdf_files": ["pyrimidine.sdf", "pyrimidine+.sdf"],
    },
    "pyridine": {
        "SMARTS": "[$(n1ccccc1),$([n+]1ccccc1)]",  # Assumes the same constant for pyridine at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": 0.5,
        "sdf_files": ["pyridine.sdf", "pyridine+.sdf"],
    },
    "pyrones": {
        "SMARTS": "[$([#8]=[#6]1:[#6]:[#6]:[#6]:[#6]:[#8]:1),$([#8]=[#6]1:[#6]:[#6]:[#8]:[#6]:[#6]:1)]",  # According to the Bain et al. article the same value for alpha- and gamma-pyrone isomers. This must be noted in Software's MANUAL.
        "constitutive_corr": -1.4,
        "sdf_files": ["gamma-pyrone.sdf", "alpha-pyrone.sdf"],
    },
    "pyrrole": {
        "SMARTS": "[$(n1cccc1),$([n-]1cccc1)]",  # Assumes the same constant for pyrrole at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": -3.5,
        "sdf_files": ["pyrrole.sdf", "pyrrole-.sdf"],
    },
    "pyrrolidine": {
        "SMARTS": "[$(N1CCCC1),$([N+]1CCCC1),$([N-]1CCCC1)]",  # Assumes the same constant for pyrrole at different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": 0.0,
        "sdf_files": ["pyrrolidine.sdf", "pyrrolidine-.sdf", "pyrrolidine+.sdf"],
    },
    "tetrahydrofuran": {
        "SMARTS": "O1CCCC1",
        "constitutive_corr": 0.0,
        "sdf_files": ["tetrahydrofuran.sdf"],
    },
    "thiazole": {
        "SMARTS": "[$(n1cscc1),$([n+]1cscc1)]",  # Assumes the same constant for thiazole on its different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": -3.0,
        "sdf_files": ["thiazole.sdf", "thiazole+.sdf"],
    },
    "thiophene": {
        "SMARTS": "s1cccc1",
        "constitutive_corr": -7.0,
        "sdf_files": ["thiophene.sdf"],
    },
    "triazine": {
        "SMARTS": "[$(n1cncnc1),$([nH+]1cncnc1),$(n1c[nH+]cnc1),$(n1cnc[nH+]c1)]",  # Assumes the same constant for 1,3,5-triazine at its different protonation states. This must be noted in Software's MANUAL.
        "constitutive_corr": -1.4,
        "sdf_files": ["1,3,5-triazine.sdf", "1,3,5-triazine+.sdf"],
    },
}

In [189]:
import json

compound: MBCompound = SDFLoader.Load(
    "1,3,5-triazine+.sdf", subdir=DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR
)

mol = compound.GetMols(to_rdkit=True)[0]
mol = RemoveHs(mol)
print(MolToSmarts(mol, isomericSmiles=True))

for idx, (key, rbt) in enumerate(RELEVENT_BOND_TYPES.items()):
    if mol.HasSubstructMatch(Chem.MolFromSmarts(rbt["SMARTS"])):
        print(f'{idx}: Match: "{key}": {json.dumps(rbt, indent=4)}')
    # else:
    #     print(f"{idx}: Didn't match: {key}")

[#6]1:[#7]:[#6]:[#7]:[#6]:[#7H+]:1
61: Match: "triazine": {
    "SMARTS": "[$(n1cncnc1),$([nH+]1cncnc1),$(n1c[nH+]cnc1),$(n1cnc[nH+]c1)]",
    "constitutive_corr": -1.4,
    "sdf_files": [
        "1,3,5-triazine.sdf",
        "1,3,5-triazine+.sdf"
    ]
}
